# PYroMat Tutorial

## Importing the package

Once PYroMat is [installed](http://www.pyromat.org/download.html), the package should be available at the Python command line as `pyromat`.

In [52]:
import pyromat as pm

Once PYroMat (`pm` for short) is imported, the `get` fuction will retrieve objects that represent individual species. For example, let's look at diatomic nitrogen, oxygen, and carbon dioxide.

In [53]:
N2 = pm.get('ig.N2')

In [54]:
O2 = pm.get('ig.O2')

In [55]:
CO2 = pm.get('ig.CO2')

The argument to the `get` function is the species ID. It is comprised of two parts: the chemical compound, e.g. "N2," and the data collection to which it belongs, "ig," for "ideal gas."

## Retrieving data

The objects expose all the methods we need to access their properties. For example, this code retrieves molecular weight, enthalpy in kJ/kg, and entropy in kJ/kg/K. We'll talk more about units in just a moment.

In [56]:
N2.mw()

28.01348

In [57]:
N2.h()

array([3588.07049914])

In [58]:
N2.s()

array([111.98585527])

But wait! Enthalpy and entropy are functions of temperature and pressure!

All properties accept [keyword arguments](https://docs.python.org/3/tutorial/controlflow.html#keyword-arguments) that allow users to define the state in terms of a set of primary properties. For ideal gases, that means temperature, `T`, pressure, `p`, or density, `d`. For a multi-phase mixture, users can also specify quality,`x`.

The command-line examples below all return the enthalpy of diatomic nitrogen at 496.5K and 3bar, but using different approaches to calling the enthalpy function.

In [59]:
N2.h(T=496.5, p=3.)     # keywords T and p **best**

array([6840.25657575])

In [60]:
N2.h(T=496.5)           # p reverts to its default

array([6840.25657575])

In [61]:
N2.h(496.5, 3.)         # T and p as ordered args

array([6840.25657575])

In [62]:
N2.h(496.5)             # p reverts to its default

array([6840.25657575])

In [63]:
N2.h(T=496.5, d=2.0358) # density instead of pressure

array([6840.25657575])

In general, PYroMat is intended to be used with keywords and values specified, but it is also designed to do its best to interpret your meaning when you leave things out. In the above examples, when the keywords are missing or when one of the properties is missing, the method is reverted to defaults. When no keywords are specified, PYroMat assumes you are working in temperature and pressure `(T,p)` tuples. When one (or both) is omitted, it reverts to a default values (see the Configuration chapter of the [PYroMat Handbook](http://www.pyromat.org/pdf/handbook.pdf) for more information).

This is a good point to note that the `h()` method returns a Numpy array instead of an ordinary float. All properties accept array-like arguments to temperature and pressure. PYroMat uses Numpy's [array broadcasting rules](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.broadcasting.html) for operating on bulk data. This can come in handy for simulations where T and p might be big [multi-dimensional arrays](http://www.pyromat.org/doc_intro.html#arrays) in time and space.

## In-line documentation

The in-line documentation describes the properties that are available and names the units used. For more detail on any individual property, just call up the in-line documentation for that method.

In [64]:
help(N2)

Help on ig2 in module builtins:

ig2, ig.N2
    The supporting data for this object were adapted from:
    B. McBride, S. Gordon, M. Reno, "Coefficients for Calculating 
    Thermodynamic and Transport Properties of Individual Species," NASA 
    Technical Memorandum 4513, 1993.



In [65]:
help(N2.h)

Help on method h:

h(*varg, **kwarg) method of builtins.ig2 instance
    Enthalpy
        h(T)   OR  h(p=p, d=d)
    
    Accepts any combination of state parameters that permit the calculation
    of temperature.  Returns the enthalpy.  
    
    Missing parameters will calculated from PYroMat's default temperature
    and pressure values in config['def_T'] and config['def_p'].
    
    Temperature in      [unit_temperature]
    pressure in         [unit_pressure]
    density in          [unit_matter / unit_volume]
    Returns enthalpy in [unit_energy / unit_matter]



## Working with units

By default, all energy is in kJ, and intensive properties are by mass (in kg). Volume is in cubic meters, and molar units are in kmols. Pressure is in bar and temperature is in Kelvin. These units are pretty broadly used, but if developing thermodynamic codes teaches us anything, it is that no system of units is pleasing to every audience. PYroMat has a configuration object that, among other things, allows the user to change the system of units.

In [66]:
N2.s(T=700.,p=50.)

array([101.00548051])

In [67]:
pm.config['unit_temperature'] = 'F'

In [68]:
N2.s(T=800.33,p=50.)*1.8    # 800.33F == 700K

array([184.34349333])

To know what the current settings are, just print the `pm.config` object.

In [69]:
pm.config

     config_file : ['/opt/anaconda3/lib/python3.9/site-packages/pyrom...
  config_verbose : False
         dat_dir : ['/opt/anaconda3/lib/python3.9/site-packages/pyrom...
 dat_exist_fatal : False
   dat_overwrite : True
   dat_recursive : True
     dat_verbose : False
           def_T : 298.15
           def_p : 1.01325
     install_dir : '/opt/anaconda3/lib/python3.9/site-packages/pyromat'
         reg_dir : ['/opt/anaconda3/lib/python3.9/site-packages/pyrom...
 reg_exist_fatal : False
   reg_overwrite : True
     reg_verbose : False
     unit_energy : 'kJ'
      unit_force : 'N'
     unit_length : 'm'
       unit_mass : 'kg'
     unit_matter : 'kmol'
      unit_molar : 'kmol'
   unit_pressure : 'bar'
unit_temperature : 'F'
       unit_time : 's'
     unit_volume : 'm3'
         version : '2.1.10'

Most of these are self-explanatory, but the subtle distinctions between `unit_mass`, `unit_molar`, and `unit_matter` definitely deserves some explanation. Mass and molar specify how mass and mole count will be specified in properties like molecular weight, but matter specifies how extensive properties like entropy and enthalpy will be reported. Unit matter can be in mass or molar units.

In [70]:
N2.mw()

28.01348

In [71]:
N2.s()

array([111.98585527])

In [72]:
pm.config['unit_matter'] = 'kmol'

In [73]:
N2.s()

array([111.98585527])

In [74]:
N2.s()/N2.mw()

array([3.99757029])

For a list of all units supported, the `units` module offers the `show()` function. This shows a space separated list of every string argument that the unit configuration parameters will accept. Matter is not shown because it can be either mass or molar units.

In [75]:
pm.units.show()

AttributeError: 'dict' object has no attribute 'iteritems'

## Working with arrays

PYroMat natively supports [Numpy](https://numpy.org) arrays. Temperature and pressure arguments can be any array-like iterable.

In [77]:
import numpy as np

In [97]:
T = np.arange(100., 1100.,100.)

In [98]:
N2.h(T)

array([  372.19966384,  1992.48522539,  3618.21383472,  5252.39930663,
        6898.08254907,  8558.18278606, 10235.34878061, 11931.81005763,
       13649.22812681, 15388.54770558])

When parameters are mixed together, they must obey Numpy's rules for array [broadcasting](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.broadcasting.html). For example, when T is a list and p is a scalar, the same pressure will be used at each temperature specified.

In [107]:
T = np.array([500., 550., 600.])

In [108]:
T

array([500., 550., 600.])

In [100]:
p = 40.5

In [101]:
N2.s(T,p)

array([ 98.82110412,  99.66223501, 100.46652165])

If we repeat the same steps, but with multiple pressures, we run into trouble. What would it mean, anyway, to have three temperatures and two pressures?

In [102]:
p = np.array([40.5, 50.])

In [103]:
N2.s(T,p)

ValueError: shape mismatch: objects cannot be broadcast to a single shape

If T and p are to be arrays, they must be exactly the same shape or they must proceed in different dimensions.

In [109]:
T = T.reshape((T.size,1))

In [110]:
N2.s(T,p)

array([[ 98.82110412,  97.84775294],
       [ 99.66223501,  98.68888382],
       [100.46652165,  99.49317047]])

In [111]:
T

array([[500.],
       [550.],
       [600.]])

In [112]:
p

array([40.5, 50. ])

This is just like populating a table for each combination of the temperature and pressure values. There are other funny multi-dimensional combinations that are supported, but we'll leave that to the Numpy documentation to cover.

Authored By:<br>
Christopher R. Martin, Ph.D.<br>
Associate Professor of Mechanical Engineering<br>
The Pennsylvania State University, Altoona College<br>
crm28@psu.edu<br>
©2021 Released under the GPLv3 License